<a href="https://colab.research.google.com/github/toughhyeok/sample-graph-chat-ui/blob/main/KeyKGRL_Exercise1_Hands_on_Practice_of_an_Inductive_KGRL_Method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
# Environment Setup
!pip install numpy==1.23.4
!pip install python-igraph==0.10.2
!pip install torch==1.13.0+cu117 --extra-index-url https://download.pytorch.org/whl/cu117
!pip install tqdm==4.64.1
!pip install scipy==1.9.3
!pip install matplotlib==3.6.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 50.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.24 requires numpy>=1.24.4, but you have numpy 1.23.4 which is incompatible.
albumentations 2.0.8 requires numpy>=1.24.4, but you have numpy 1.23.4 which is incompatible.
bigframes 2.10.0 requires numpy>=1.24.0, but you have numpy 1.23.4 which is incompatible.
blosc2 3.5.1 requires numpy>=1.26, but you have numpy 1.23.4 which is incompatible.
chex 0.1.89 requires numpy>=1.24.1, but you have numpy 1.23.4 which is incompatible.
db-dtypes 1.4.3 requires numpy>=1.24.0, but you have numpy 1.23.4 which is incompatible.
imbalanced-learn 0.13.0 requires numpy<3,>=1.24.3, but you have num

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 35.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 1.3 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.6.0+cu124
    Uninstalling torch-2.6.0+cu124:
      Successfully uninstalled torch-2.6.0+cu124
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
accelerate 1.8.1 requires torch>=2.0.0, but you have torch 1.13.0+cu117 which is incompatible.
torchaudio 2.6.0+cu124 requires torch==2.6.0, but you have torch 1.13.0+cu117 which is incompatible.
torchdata 0.11.0 requires torch>=2, but you have torch 1.13.0+cu117 which is incompatible.
torchvision 0.21.0+cu124 requires torch==2.6.0, but you have torch 1.13.0+cu117 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Clone Official Repository of InGram
!git clone https://github.com/bdi-lab/InGram.git
%cd InGram

Cloning into 'InGram'...
remote: Enumerating objects: 156, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 156 (delta 51), reused 32 (delta 32), pack-reused 82 (from 1)
Receiving objects: 100% (156/156), 15.59 MiB | 5.83 MiB/s, done.
Resolving deltas: 100% (75/75), done.
/content/InGram


In [ ]:
# Download InGram's Checkpoints and unzip
!gdown https://drive.google.com/uc?id=1aZrx2dYNPT7j4TGVBOGqHMdHRwFUBqx5 -O ckpt.zip
!unzip ckpt.zip

Downloading...
From (original): https://drive.google.com/uc?id=1aZrx2dYNPT7j4TGVBOGqHMdHRwFUBqx5
From (redirected): https://drive.google.com/uc?id=1aZrx2dYNPT7j4TGVBOGqHMdHRwFUBqx5&confirm=t&uuid=d25e3b21-3d31-48f2-9ca5-725da487068b
To: /content/InGram/ckpt.zip
100% 122M/122M [00:02<00:00, 43.0MB/s]
Archive:  ckpt.zip
   creating: ckpt/best/
   creating: ckpt/best/FB-100/
  inflating: ckpt/best/FB-100/best.ckpt  
  inflating: ckpt/best/FB-100/config.json  
   creating: ckpt/best/FB-25/
  inflating: ckpt/best/FB-25/best.ckpt  
  inflating: ckpt/best/FB-25/config.json  
   creating: ckpt/best/FB-50/
  inflating: ckpt/best/FB-50/best.ckpt  
  inflating: ckpt/best/FB-50/config.json  
   creating: ckpt/best/FB-75/
  inflating: ckpt/best/FB-75/best.ckpt  
  inflating: ckpt/best/FB-75/config.json  
   creating: ckpt/best/NELL-995-v1/
  inflating: ckpt/best/NELL-995-v1/best.ckpt  
  inflating: ckpt/best/NELL-995-v1/config.json  
   creating: ckpt/best/NL-0/
  inflating: ckpt/best/NL-0/best.ckp

In [ ]:
# Import Modules
from relgraph import generate_relation_triplets
from dataset import TestNewData
from tqdm import tqdm
import random
from model import InGram
import torch
import numpy as np
from utils import get_rank, get_metrics
from evaluation import evaluate
import matplotlib.pyplot as plt
import json

In [ ]:
# Fix Random Seeds
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

In [ ]:
data_name = "FB-100" ## We use FB-100 in this exercise

In [ ]:
# Load Hyperparameters
with open(f"./ckpt/best/{data_name}/config.json") as f:
    configs = json.load(f)

# Load Dataset
test = TestNewData(f"./data/{data_name}/", data_type = "test")

# Load Model & Checkpoint

my_model = InGram(dim_ent = configs['dimension_entity'], hid_dim_ratio_ent = configs['hidden_dimension_ratio_entity'],
                  dim_rel = configs['dimension_relation'], hid_dim_ratio_rel = configs['hidden_dimension_ratio_relation'],
                  num_bin = configs['num_bin'], num_layer_ent = configs['num_layer_ent'], num_layer_rel = configs['num_layer_rel'],
                  num_head = configs['num_head'])
my_model = my_model.cuda()

my_model.load_state_dict(torch.load(f"ckpt/best/{data_name}/best.ckpt")["model_state_dict"])


-----Test Data Statistics-----
Message set has 6987 triplets
Supervision set has 2329 triplets
2624 entities, 77 relations, 11645 triplets


<All keys matched successfully>

# Inspecting a Relation Graph

In [ ]:
# Create a Relation Graph
test_relation_triplets = generate_relation_triplets(test.msg_triplets, test.num_ent, test.num_rel, configs['num_bin'])

## Relations that have the highest/lowest affinity scores with relation "/media_common/netflix_genre/titles"

In [ ]:
target_id = test.rel2id['/media_common/netflix_genre/titles']


print("===========Relations that have the highest affinity scores with \"/media_common/netflix_genre/titles\"===========")
for r1,r2,s12 in test_relation_triplets:
  if r1 == target_id and s12 == 0:
    print(test.id2rel[r2%test.num_rel]+ ("_inv" if r2>=test.num_rel else ""))

===========Relations that have the highest affinity scores with "/media_common/netflix_genre/titles"===========
/media_common/netflix_genre/titles
/film/film/release_date_s./film/film_regional_release_date/film_release_distribution_medium_inv
/film/film/other_crew./film/film_crew_gig/film_crew_role_inv
/film/film/genre_inv


In [ ]:
print("===========Relations that have the lowest affinity scores with \"/media_common/netflix_genre/titles\"===========")
for r1,r2,s12 in test_relation_triplets:
  if r1 == target_id and s12 == configs['num_bin']-1:
    print(test.id2rel[r2%test.num_rel]+ ("_inv" if r2>=test.num_rel else ""))

===========Relations that have the lowest affinity scores with "/media_common/netflix_genre/titles"===========
/sports/sports_team_location/teams
/location/statistical_region/religions./location/religion_percentage/religion
/base/aareas/schema/administrative_area/administrative_parent
/location/statistical_region/places_exported_to./location/imports_and_exports/exported_to
/military/military_combatant/military_conflicts./military/military_combatant_group/combatants
/location/country/capital
/olympics/olympic_participating_country/athletes./olympics/olympic_athlete_affiliation/olympics
/location/country/form_of_government
/base/aareas/schema/administrative_area/administrative_area_type
/people/person/places_lived./people/place_lived/location_inv
/people/person/spouse_s./people/marriage/location_of_ceremony_inv
/base/culturalevent/event/entity_involved_inv
/location/statistical_region/religions./location/religion_percentage/religion_inv
/base/aareas/schema/administrative_area/administrat

## Exercise: choose a relation and find the relations that have the highest/lowest affinity scores

In [ ]:
## List of relations
print("===========List of Relations===========")
for rel_name in test.id2rel:
  print(rel_name)
  print(rel_name+"_inv")

===========List of Relations===========
/common/topic/webpage./common/webpage/category
/common/topic/webpage./common/webpage/category_inv
/people/person/religion
/people/person/religion_inv
/film/film/release_date_s./film/film_regional_release_date/film_release_distribution_medium
/film/film/release_date_s./film/film_regional_release_date/film_release_distribution_medium_inv
/award/award_winning_work/awards_won./award/award_honor/award_winner
/award/award_winning_work/awards_won./award/award_honor/award_winner_inv
/film/film/other_crew./film/film_crew_gig/film_crew_role
/film/film/other_crew./film/film_crew_gig/film_crew_role_inv
/film/film/genre
/film/film/genre_inv
/music/genre/artists
/music/genre/artists_inv
/people/cause_of_death/people
/people/cause_of_death/people_inv
/location/location/contains
/location/location/contains_inv
/people/person/places_lived./people/place_lived/location
/people/person/places_lived./people/place_lived/location_inv
/media_common/netflix_genre/titles
/

In [ ]:
## Choose a relation
target_rel = ""

#target_rel = "/media_common/netflix_genre/titles_inv"


In [ ]:
print(f"===========Relations that have the highest affinity scores with \"{target_rel}\"===========")
if "_inv"!= target_rel[-4:]:
  target_id = test.rel2id[target_rel]
else:
  target_id = test.rel2id[target_rel[:-4]]+test.num_rel
for r1,r2,s12 in test_relation_triplets:
  if r1 == target_id and s12 == 0:
    print(test.id2rel[r2%test.num_rel]+ ("_inv" if r2>=test.num_rel else ""))

===========Relations that have the highest affinity scores with "/media_common/netflix_genre/titles"===========
/media_common/netflix_genre/titles
/film/film/release_date_s./film/film_regional_release_date/film_release_distribution_medium_inv
/film/film/other_crew./film/film_crew_gig/film_crew_role_inv
/film/film/genre_inv


In [ ]:
print(f"===========Relations that have the lowest affinity scores with \"{target_rel}\"===========")
for r1,r2,s12 in test_relation_triplets:
  if r1 == target_id and s12 == configs['num_bin']-1:
    print(test.id2rel[r2%test.num_rel]+ ("_inv" if r2>=test.num_rel else ""))

===========Relations that have the lowest affinity scores with "/media_common/netflix_genre/titles"===========
/sports/sports_team_location/teams
/location/statistical_region/religions./location/religion_percentage/religion
/base/aareas/schema/administrative_area/administrative_parent
/location/statistical_region/places_exported_to./location/imports_and_exports/exported_to
/military/military_combatant/military_conflicts./military/military_combatant_group/combatants
/location/country/capital
/olympics/olympic_participating_country/athletes./olympics/olympic_athlete_affiliation/olympics
/location/country/form_of_government
/base/aareas/schema/administrative_area/administrative_area_type
/people/person/places_lived./people/place_lived/location_inv
/people/person/spouse_s./people/marriage/location_of_ceremony_inv
/base/culturalevent/event/entity_involved_inv
/location/statistical_region/religions./location/religion_percentage/religion_inv
/base/aareas/schema/administrative_area/administrat

# Reproducing the Results of InGram

In [ ]:
with torch.no_grad():
    my_model.eval()
    msg = torch.tensor(test.msg_triplets).cuda()
    sup = torch.tensor(test.sup_triplets).cuda()
    relation_triplets = torch.tensor(test_relation_triplets).cuda()
    test_init_emb_ent = torch.load(f"ckpt/best/{data_name}/best.ckpt")["inf_emb_ent"]
    test_init_emb_rel = torch.load(f"ckpt/best/{data_name}/best.ckpt")["inf_emb_rel"]
    emb_ent, emb_rel = my_model(test_init_emb_ent, test_init_emb_rel, msg, relation_triplets)

    head_ranks = []
    tail_ranks = []
    ranks = []
    for triplet in tqdm(sup):
        triplet = triplet.unsqueeze(dim = 0)

        head_corrupt = triplet.repeat(test.num_ent, 1)
        head_corrupt[:,0] = torch.arange(end = test.num_ent)

        head_scores = my_model.score(emb_ent, emb_rel, head_corrupt)
        head_filters = test.filter_dict[('_', int(triplet[0,1].item()), int(triplet[0,2].item()))]
        head_rank = get_rank(triplet, head_scores, head_filters, target = 0)

        tail_corrupt = triplet.repeat(test.num_ent, 1)
        tail_corrupt[:,2] = torch.arange(end = test.num_ent)

        tail_scores = my_model.score(emb_ent, emb_rel, tail_corrupt)
        tail_filters = test.filter_dict[(int(triplet[0,0].item()), int(triplet[0,1].item()), '_')]
        tail_rank = get_rank(triplet, tail_scores, tail_filters, target = 2)

        ranks.append(head_rank)
        head_ranks.append(head_rank)
        ranks.append(tail_rank)
        tail_ranks.append(tail_rank)


    print("\n========LP========")
    mr, mrr, hit10, hit3, hit1 = get_metrics(ranks)
    print(f"MR: {mr:.1f}")
    print(f"MRR: {mrr:.3f}")
    print(f"Hits@10: {hit10:.3f}")
    print(f"Hits@1: {hit1:.3f}")

/content/InGram/model.py:117: UserWarning: scatter_reduce() is in beta and the API may change at any time. (Triggered internally at ../aten/src/ATen/native/TensorAdvancedIndexing.cpp:1615.)
  attn_val_max = torch.zeros((num_rel, self.num_head, 1)).cuda().scatter_reduce(dim = 0, \
100%|██████████| 2329/2329 [00:03<00:00, 653.13it/s]


========LP========
MR: 171.5
MRR: 0.223
Hits@10: 0.371
Hits@1: 0.146
